In [1]:
import os
import shutil

log_dir = "logs/hparam_tuning"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

os.makedirs(log_dir, exist_ok=True)

%load_ext tensorboard
%tensorboard --logdir logs/hparam_tuning

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

print("TensorFlow version:", tf.__version__)

physical_devices = tf.config.list_physical_devices('GPU')
print("Available GPUs:", physical_devices)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPU(s) are available for use.")
    except RuntimeError as e:
        print(e)



ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-12-11 22:09:13.373814: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 22:09:13.417365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 22:09:13.470962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 22:09:13.484442: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registere

2024-12-11 22:09:22.339489: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 22:09:22.354459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 22:09:22.374115: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 22:09:22.378784: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 22:09:22.390146: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.17.0
Available GPUs: []


I0000 00:00:1733944166.372323  109701 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-11 22:09:26.433340: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
data = pd.read_csv("consecutive_feed.csv", delimiter=',')
df = data[["SAMAN", "ARPA", "SLAJ", "YEM", "SU"]]
print(df.head())

df['SAMAN_derivative'] = np.gradient(df['SAMAN'])
df['ARPA_derivative'] = np.gradient(df['ARPA'])
df['SLAJ_derivative'] = np.gradient(df['SLAJ'])
df['YEM_derivative'] = np.gradient(df['YEM'])
df['SU_derivative'] = np.gradient(df['SU'])



      SAMAN      ARPA      SLAJ       YEM        SU
0  1.623454  2.272836  2.516354  6.818507  1.298763
1  1.543873  2.161422  2.393003  6.484267  1.235098
2  1.378363  2.297272  2.909877  6.126058  1.225212
3  1.378363  2.297272  2.909877  6.126058  1.225212
4  1.378363  2.297272  2.909877  6.126058  1.225212


In [3]:
from tensorboard.plugins.hparams import api as hp

HP_WINDOW_SIZE = hp.HParam('window_size', hp.Discrete([3, 5]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.3]))
HP_L2 = hp.HParam('l2', hp.Discrete([0.0001, 0.001]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.005]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64]))

METRIC_RMSE = 'rmse'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[
            HP_WINDOW_SIZE, HP_DROPOUT, HP_L2, HP_OPTIMIZER, HP_LEARNING_RATE, HP_BATCH_SIZE
        ],
        metrics=[hp.Metric(METRIC_RMSE, display_name='RMSE')],
    )



In [4]:
def df_to_X_y(df, var_to_guess_1, var_to_guess_2, WINDOW_SIZE):
    df_as_np = df.to_numpy()
    X = []
    y = []

    var_to_guess_1_index = df.columns.get_loc(var_to_guess_1)
    var_to_guess_2_index = df.columns.get_loc(var_to_guess_2)

    for i in range(len(df) - WINDOW_SIZE):
        X.append(df_as_np[i:i+WINDOW_SIZE, :])
        var_1_value = df_as_np[i+WINDOW_SIZE, var_to_guess_1_index]
        var_2_value = df_as_np[i+WINDOW_SIZE, var_to_guess_2_index]
        y.append([var_1_value, var_2_value])
    return np.array(X), np.array(y)

def prepare_data(df, var_to_guess_1, var_to_guess_2, WINDOW_SIZE):

    split_1 = int(0.8 * len(df))
    split_2 = int(0.9 * len(df))
    df_train = df[:split_1]
    df_val = df[split_1:split_2]
    df_test = df[split_2:]

    scalers = {}
    scaled_train = pd.DataFrame()
    scaled_val = pd.DataFrame()
    scaled_test = pd.DataFrame()

    for column in df.columns:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_train[column] = scaler.fit_transform(df_train[[column]]).ravel()
        scaled_val[column] = scaler.transform(df_val[[column]]).ravel()
        scaled_test[column] = scaler.transform(df_test[[column]]).ravel()
        scalers[column] = scaler

    X_train, y_train = df_to_X_y(scaled_train, var_to_guess_1, var_to_guess_2, WINDOW_SIZE)
    X_val, y_val = df_to_X_y(scaled_val, var_to_guess_1, var_to_guess_2, WINDOW_SIZE)
    X_test, y_test = df_to_X_y(scaled_test, var_to_guess_1, var_to_guess_2, WINDOW_SIZE)

    return X_train, y_train, X_val, y_val, X_test, y_test

def train_test_model(hparams, run_name):
    var_to_guess_1 = "YEM"
    var_to_guess_2 = "SLAJ"

    WINDOW_SIZE = hparams[HP_WINDOW_SIZE]
    X_train, y_train, X_val, y_val, X_test, y_test = prepare_data(df, var_to_guess_1, var_to_guess_2, WINDOW_SIZE)

    model = Sequential()
    model.add(InputLayer(input_shape=(WINDOW_SIZE, X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(hparams[HP_L2])))
    model.add(Dropout(hparams[HP_DROPOUT]))
    model.add(LSTM(32, kernel_regularizer=l2(hparams[HP_L2])))
    model.add(BatchNormalization())
    model.add(Dropout(hparams[HP_DROPOUT]))
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(hparams[HP_L2])))
    model.add(Dense(2, activation='linear'))

    if hparams[HP_OPTIMIZER] == 'adam':
        optimizer = Adam(learning_rate=hparams[HP_LEARNING_RATE])
    elif hparams[HP_OPTIMIZER] == 'sgd':
        optimizer = SGD(learning_rate=hparams[HP_LEARNING_RATE])

    model.compile(loss='huber', optimizer=optimizer, metrics=[RootMeanSquaredError()])


    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(log_dir, run_name), 
        histogram_freq=1
    )


    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=5, min_lr=1e-7, verbose=0)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)


    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=hparams[HP_BATCH_SIZE],
        callbacks=[reduce_lr, early_stopping, tensorboard_callback],  
        verbose=0
    )


    test_loss, test_rmse = model.evaluate(X_test, y_test, verbose=0)
    return test_rmse


def run(run_dir, hparams, run_name):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  
        test_rmse = train_test_model(hparams, run_name)
        tf.summary.scalar(METRIC_RMSE, test_rmse, step=1)




In [ ]:

session_num = 0

for window_size in HP_WINDOW_SIZE.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for l2_reg in HP_L2.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                for learning_rate in HP_LEARNING_RATE.domain.values:
                    for batch_size in HP_BATCH_SIZE.domain.values:
                        hparams = {
                            HP_WINDOW_SIZE: window_size,
                            HP_DROPOUT: dropout_rate,
                            HP_L2: l2_reg,
                            HP_OPTIMIZER: optimizer,
                            HP_LEARNING_RATE: learning_rate,
                            HP_BATCH_SIZE: batch_size,
                        }
                        run_name = f"run-{session_num}"
                        print(f'--- Starting trial: {run_name}')
                        print({h.name: hparams[h] for h in hparams})
                        
                        run('logs/hparam_tuning/' + run_name, hparams, run_name)
                        
                        session_num += 1






--- Starting trial: run-0
{'window_size': 3, 'dropout': 0.1, 'l2': 0.0001, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 32}


/home/erincada/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


--- Starting trial: run-1
{'window_size': 3, 'dropout': 0.1, 'l2': 0.0001, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 64}


/home/erincada/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


--- Starting trial: run-2
{'window_size': 3, 'dropout': 0.1, 'l2': 0.0001, 'optimizer': 'adam', 'learning_rate': 0.005, 'batch_size': 32}


...

/home/erincada/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


--- Starting trial: run-24
{'window_size': 3, 'dropout': 0.3, 'l2': 0.001, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 32}


/home/erincada/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


--- Starting trial: run-25
{'window_size': 3, 'dropout': 0.3, 'l2': 0.001, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 64}


/home/erincada/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
